# Symbolic Transformer

This project is intended to support describing and reasoning about the internals of Transformer Language Model algorithms - keeping a close link between code and notation. Initial work is based on the Pythia models from Eleuther AI and Neel Nandas Transformer Lens analysis tool. 

# Terminology and Conventions

Analysis is focussed on individual residual streams, that is the vectors at a given position in particular layers of a transformer. See descriptions in https://transformer-circuits.pub/2021/framework/index.html and https://www.neelnanda.io/mechanistic-interpretability/glossary

The notation aims to be consistent with that described in https://transformer-circuits.pub/2021/framework/index.html#notation but there are deviations.

A transformer block is the combination of attention layer and MLP. Superscripts will generally refer to the block number, with input tokens starting at block 0 before the first transformer block, with blocks numbered from 1 to the number of blocks for the model (6 for pythia-70m-deduped).

$W_E$ represents the token embedding matrix.
$W_U$ represents the unembedding matrix.

The position refers to the ordering of tokens within the model context window. The first token has position 0, the last has $\#\text{context window length} - 1$ (2047 is the last position for pythia-70m-deduped). The position number is generally indicated with a subscript. 

Vectors from applying the embedding layer to a particular token (token vectors) are labeled with that token underlined - e.g. $\underline{\text{The}}$. This can be combined with position subscripts, indicating that the vector has been combined with position. Token layer superscripts when emphasising flow of information from that token vector through the transformer.

More generally the residual vector after block $i$, at position $j$ for a given input is written as $x^i_j$. 

The vectors relating to a particular token in the unembedding matrix is the token text with an overline.

The contribution from a particular block to the residual stream is written as $\Delta x^2_1$ or $\Delta \underline{\text{the}}^2_1$. The structure is model dependent and probably block dependent, for pythia each block can be decomposed into Layer Norm, Attention and MLP.

The attention pattern of head 3 in layer 4 is $A^{4,3}$.

The MLP layer Layer as an operator is "$M^2$. 

Layer Normalization as an operator is $LN$.

# Analysis

When inference is run, the unembedding weighs are applied to the residual vectors from the final layer to 
generate a logit (i.e. an unbounded output weight which can be normalized across all tokens to give each a
probabilty between 0 and 1). The predicted token is selected from those with the greatest logits. During training
the loss for gradient descent is calculated based on logit of the actual next token.

With output residual vector $x_0^6$ this can be written as
$$
W_U \ket{x_0^6}
$$

The loss for a particular token is the dot product of the row for that token id (say '$\overline{ublin}$') with the output residual:

$$
<\overline{\text{ublin}}_0^6 , x_0^6 >
$$

The layers of the transformer add to the residual stream, so the output is based on

$$
x_0^6 = LN ( \underline{\text{D}}_0^0 + \sum_{i=0}^{6} \Delta x_0^i )
$$

where $\underline{\text{D}}_0^0$ is a row from the embedding matrix $W_E$ combined with positional embeddings, and $\Delta x_0^i$ is the contribution from block i+1 (which takes $x_o^i$ as input).

Layer Normalization is applied to the result however, and as this is a non linear operation we can't safely decompose these inputs.




# Layer Normalization and Dot Product

$$<v1,LN v2> \approx |v1| \cos{\theta_{v1,c(v2)}} $$
where $\theta_{a,b}$ is the angle between vectors $a$ and $b$, and $c(v)$ is the centering operation described in reexamine_layer_norm.ipynb

Each vector sum can be considered in 2d, where it either increases or decreases $\theta$. By building up the sum of vectors on the right we can see which vectors contribute to the angle used in the final dot product.

# Objective of SymbolicTransformer

The goal is to describe the operation of a Transformer on a token as a high level expression, with the ability to expand on terms in the expression, disregard terms with low impact etc